In [1]:
import pandas as pd
import warnings
from src.utils import *
import openai
import src.gpt_siemens as siemens
warnings.filterwarnings('ignore')

### 1) Data

In [2]:
df=pd.read_pickle('data/df_Mahle_full.pkl')

### 2) Duplicates

In [3]:
df['pair'] = df.apply(lambda row: tuple(sorted([row['Product Id'], row['matched_Product Id']])), axis=1)

no_duplicates = df[~df.duplicated(subset=['pair'], keep=False)]  # Rows with no duplicates
duplicates = df[df.duplicated(subset=['pair'], keep=False)]     # Rows with duplicates

score_same = duplicates.groupby('pair').filter(lambda x: len(x['score'].unique()) == 1)    # Duplicates with same values in the score column
score_diff_tot = duplicates.groupby('pair').filter(lambda x: len(x['score'].unique()) > 1) # Duplicates with different score values
score_diff_uniq = score_diff_tot.drop_duplicates(subset=['pair', 'score'], keep='first')   # Unique different score values

In [4]:
df.shape

(931744, 19)

Differences based on the duplicates and their scores

In [5]:
a = no_duplicates.shape[0]
b = score_same.shape[0]
c = score_diff_tot.shape[0]
total = a + b + c
print(f"No duplicates: {a}, Same scores: {b}, Different scores: {c}, Total: {total}")

No duplicates: 86492, Same scores: 734099, Different scores: 111153, Total: 931744


#### 2.1) For the rows with the same scores leave those ones with longer pid values

In [6]:
score_same.head(2)

,Product Id,Manufacturer Name,Manufacturer PID,model,Short Description,Long Description,Additional description,score,matched_pid_left,matched_pid_right,matched_brand_left,matched_brand_right,matched_Product Id,matched_model,matched_Manufacturer PID,matched_Supplier PID,matched_Manufacturer Name,matched_Supplier Name,pair
0,61085983,CARR-LANE,CL-40-BP-1,CL-40-BP-1,"PLUNGER, 1/4 X 20 PN CL-40-BP-1",NaN,NaN,85.71,cl40bp1,cl70bp1,carrlane,carrlane,61085352,CL70BP1,CL70BP1,NaN,CARR LANE,NaN,"(61085352, 61085983)"
3,61068377,Yudo,NZTPIC162150,NZTPIC162150,THERMOCOUPLE,NaN,NaN,91.67,nztpic162150,nztpic162850,yudo,yudo,61072581,NZTPIC162850,NZTPIC162850,NaN,Yudo,NaN,"(61068377, 61072581)"


In [7]:
score_same[score_same['pair']==(61085352, 61085983)]

,Product Id,Manufacturer Name,Manufacturer PID,model,Short Description,Long Description,Additional description,score,matched_pid_left,matched_pid_right,matched_brand_left,matched_brand_right,matched_Product Id,matched_model,matched_Manufacturer PID,matched_Supplier PID,matched_Manufacturer Name,matched_Supplier Name,pair
0,61085983,CARR-LANE,CL-40-BP-1,CL-40-BP-1,"PLUNGER, 1/4 X 20 PN CL-40-BP-1",NaN,NaN,85.71,cl40bp1,cl70bp1,carrlane,carrlane,61085352,CL70BP1,CL70BP1,NaN,CARR LANE,NaN,"(61085352, 61085983)"
701395,61085352,CARR LANE,CL70BP1,CL70BP1,"PLUNGER, CARR LANE CL-70-BP-1 BALL NOSE",NaN,NaN,85.71,cl70bp1,cl40bp1,carrlane,carrlane,61085983,CL-40-BP-1,CL-40-BP-1,NaN,CARR-LANE,NaN,"(61085352, 61085983)"


The lowest length value is 5

In [8]:
length_below_6 = score_same[(score_same['length_left'] < 6) | (score_same['length_right'] < 6)]
length_below_6.shape

KeyError: 'length_left'

In [12]:
length_below_6.length_left.value_counts()

length_left
5    38118
6     2128
7      509
Name: count, dtype: int64

In [13]:
length_below_6.length_right.value_counts()

length_right
5    37039
6     1791
4     1597
7      328
Name: count, dtype: int64

We keep the longest pid values

In [14]:
score_same['length_left'] = score_same['matched_pid_left'].astype(str).str.len()
score_same['length_right'] = score_same['matched_pid_right'].astype(str).str.len()

score_same['total_length'] = score_same['length_left'] + score_same['length_right']   # Calculate total length
no_duplic_same_score = score_same.loc[score_same.groupby('pair')['total_length'].idxmax()]

In [15]:
no_duplic_same_score[no_duplic_same_score['pair']==(61085352, 61085983)]

,Product Id,Manufacturer Name,Manufacturer PID,model,Short Description,Long Description,Additional description,score,matched_pid_left,matched_pid_right,...,matched_Product Id,matched_model,matched_Manufacturer PID,matched_Supplier PID,matched_Manufacturer Name,matched_Supplier Name,pair,length_left,length_right,total_length
0,61085983,CARR-LANE,CL-40-BP-1,CL-40-BP-1,"PLUNGER, 1/4 X 20 PN CL-40-BP-1",NaN,NaN,85.71,cl40bp1,cl70bp1,...,61085352,CL70BP1,CL70BP1,NaN,CARR LANE,NaN,"(61085352, 61085983)",7,7,14


No duplicates

In [16]:
duplicates_in_filtered = no_duplic_same_score[no_duplic_same_score.duplicated(subset=['pair'], keep=False)]
print(duplicates_in_filtered.shape[0])

0


#### 2.2) For rows with different score values. Keep the highest and then check the lenght of the pids.

In [17]:
score_diff_tot.head()

,Product Id,Manufacturer Name,Manufacturer PID,model,Short Description,Long Description,Additional description,score,matched_pid_left,matched_pid_right,matched_brand_left,matched_brand_right,matched_Product Id,matched_model,matched_Manufacturer PID,matched_Supplier PID,matched_Manufacturer Name,matched_Supplier Name,pair
1,61085983,CARR-LANE,CL-40-BP-1,CL-40-BP-1,"PLUNGER, 1/4 X 20 PN CL-40-BP-1",NaN,NaN,71.43,cl40bp1,cl40sbp2,carrlane,carrlane,61085952,CL-40-SBP-2,CL-40-SBP-2,NaN,CARR LANE,NaN,"(61085952, 61085983)"
2,61085983,CARR-LANE,CL-40-BP-1,CL-40-BP-1,"PLUNGER, 1/4 X 20 PN CL-40-BP-1",NaN,NaN,58.93,cl40bp1,clm10bp1,carrlane,carrline,61069715,NaN,CLM-10-BP1,NaN,CARR LINE,NaN,"(61069715, 61085983)"
6,61085951,Banner,Q23SP6RQ,Q23SP6RQ,"BANNER, Q23SP6RQ",NaN,NaN,75.00,q23sp6rq,q23sp6d,banner,banner,61098160,Q23SP6D,Q23SP6D,NaN,Banner,NaN,"(61085951, 61098160)"
13,61068180,Bosch,R165111320,NaN,BOSCH REXROTH LINEAR R165111320 SIZE 15,NaN,NaN,90.00,r165111320,165111320,bosch,bosch,60993267,NaN,1651-113-20,NaN,BOSCH REXROTH,NaN,"(60993267, 61068180)"
55,61068137,FESTO,163302,DNC-32-100-PPV-A-Q-52,Normzylinder,NaN,NaN,100.00,163302,163302,festo,festo,61003496,DNC-32-10-PPV-A-Q-S2,163302,NaN,FESTO,NaN,"(61003496, 61068137)"


In [18]:
score_diff_tot[score_diff_tot['pair']==(61085952, 61085983)]

,Product Id,Manufacturer Name,Manufacturer PID,model,Short Description,Long Description,Additional description,score,matched_pid_left,matched_pid_right,matched_brand_left,matched_brand_right,matched_Product Id,matched_model,matched_Manufacturer PID,matched_Supplier PID,matched_Manufacturer Name,matched_Supplier Name,pair
1,61085983,CARR-LANE,CL-40-BP-1,CL-40-BP-1,"PLUNGER, 1/4 X 20 PN CL-40-BP-1",NaN,NaN,71.43,cl40bp1,cl40sbp2,carrlane,carrlane,61085952,CL-40-SBP-2,CL-40-SBP-2,NaN,CARR LANE,NaN,"(61085952, 61085983)"
584546,61085952,CARR LANE,CL-40-SBP-2,CL-40-SBP-2,"BALL PLUNGER, CARR-LANE # CL-40-SBP-2",NaN,NaN,75.00,cl40sbp2,cl40bp1,carrlane,carrlane,61085983,CL-40-BP-1,CL-40-BP-1,NaN,CARR-LANE,NaN,"(61085952, 61085983)"


In [19]:
no_duplic_dif_score = score_diff_tot.loc[score_diff_tot.groupby('pair')['score'].idxmax()]

In [20]:
no_duplic_dif_score.score.describe()

count    26325.000000
mean        86.433161
std          8.057110
min         52.380000
25%         81.820000
50%         87.500000
75%         91.670000
max        100.000000
Name: score, dtype: float64

In [21]:
no_duplic_dif_score[no_duplic_dif_score['pair']==(61085952, 61085983)]

,Product Id,Manufacturer Name,Manufacturer PID,model,Short Description,Long Description,Additional description,score,matched_pid_left,matched_pid_right,matched_brand_left,matched_brand_right,matched_Product Id,matched_model,matched_Manufacturer PID,matched_Supplier PID,matched_Manufacturer Name,matched_Supplier Name,pair
584546,61085952,CARR LANE,CL-40-SBP-2,CL-40-SBP-2,"BALL PLUNGER, CARR-LANE # CL-40-SBP-2",NaN,NaN,75.0,cl40sbp2,cl40bp1,carrlane,carrlane,61085983,CL-40-BP-1,CL-40-BP-1,NaN,CARR-LANE,NaN,"(61085952, 61085983)"


In [22]:
no_duplic_dif_score['length_left'] = no_duplic_dif_score['matched_pid_left'].astype(str).str.len()
no_duplic_dif_score['length_right'] = no_duplic_dif_score['matched_pid_right'].astype(str).str.len()
length_below_6_dif_score = no_duplic_dif_score[(no_duplic_dif_score['length_left'] < 6) | (no_duplic_dif_score['length_right'] < 6)]

In [23]:
length_below_6_dif_score.length_left.value_counts()

length_left
6    1237
7     215
5     134
Name: count, dtype: int64

In [24]:
length_below_6_dif_score.length_right.value_counts()

length_right
5    1583
4       3
Name: count, dtype: int64

In [25]:
length_below_6_dif_score[length_below_6_dif_score['length_right']==4]

,Product Id,Manufacturer Name,Manufacturer PID,model,Short Description,Long Description,Additional description,score,matched_pid_left,matched_pid_right,...,matched_brand_right,matched_Product Id,matched_model,matched_Manufacturer PID,matched_Supplier PID,matched_Manufacturer Name,matched_Supplier Name,pair,length_left,length_right
307639,61024281,FESTO,130610,QSY-8-4,Y-Steckverbindung,NaN,NaN,80.0,qsy84,qs84,...,festo,61012216,QS-8-4,130606,NaN,FESTO,NaN,"(61012216, 61024281)",5,4
307638,61024281,FESTO,130610,QSY-8-4,Y-Steckverbindung,NaN,NaN,80.0,qsy84,qs84,...,festo,77140099,QS-8-4,130606,NaN,FESTO,NaN,"(61024281, 77140099)",5,4
307637,61024281,FESTO,130610,QSY-8-4,Y-Steckverbindung,NaN,NaN,80.0,qsy84,qs84,...,festo,77140301,QS-8-4,130606,NaN,FESTO,NaN,"(61024281, 77140301)",5,4


#### 2.3) All together without dupilcates

In [31]:
no_duplic_same_score=no_duplic_same_score.drop(columns=['length_left', 'length_right', 'total_length'])
no_duplic_dif_score=no_duplic_dif_score.drop(columns=['length_left', 'length_right'])

In [32]:
df_final = pd.concat([no_duplicates, no_duplic_same_score, no_duplic_dif_score], ignore_index=True)

In [33]:
df_final.head()

,Product Id,Manufacturer Name,Manufacturer PID,model,Short Description,Long Description,Additional description,score,matched_pid_left,matched_pid_right,matched_brand_left,matched_brand_right,matched_Product Id,matched_model,matched_Manufacturer PID,matched_Supplier PID,matched_Manufacturer Name,matched_Supplier Name,pair
0,61068240,ROCKWELL AUTOMATION,NaN,140M-C2E-A63,140M Motorschutzschalter,NaN,NaN,70.00,140mc2ea63,140mc2eb16,rockwellautomation,rockwellautomation,61093882,140M-C2E-B16,NaN,NaN,ROCKWELL AUTOMATION,NaN,"(61068240, 61093882)"
1,61068240,ROCKWELL AUTOMATION,NaN,140M-C2E-A63,140M Motorschutzschalter,NaN,NaN,70.00,140mc2ea63,140mc2eb16,rockwellautomation,rockwellautomation,61069420,140M-C2E-B16,NaN,NaN,ROCKWELL AUTOMATION,NaN,"(61068240, 61069420)"
2,61068240,ROCKWELL AUTOMATION,NaN,140M-C2E-A63,140M Motorschutzschalter,NaN,NaN,70.00,140mc2ea63,140mc2eb16,rockwellautomation,rockwellautomation,61062366,NaN,140M-C2E-B16,NaN,ROCKWELL AUTOMATION,NaN,"(61062366, 61068240)"
3,61068385,Yudo,CYVPTY104500,CYVPTY104500,VALVE PIN,NaN,NaN,75.00,cyvpty104500,cyvpty064501,yudo,yudo,61072643,CYVPTY064501,CYVPTY064501,NaN,Yudo,NaN,"(61068385, 61072643)"
4,61068637,SCHAEFFLER (INA/FAG),0055453740000,KR16-PP,Nadel-Kurvenrolle,NaN,NaN,83.33,kr16pp,kr16ppa,schaefflerinafag,schaefflerinafag,61011925,NaN,KR16-PP-A,NaN,SCHAEFFLER (INA/FAG),NaN,"(61011925, 61068637)"


In [34]:
df_final.score.describe()

count    240944.000000
mean         85.199746
std          10.977235
min          51.050000
25%          79.170000
50%          85.710000
75%          93.330000
max         100.000000
Name: score, dtype: float64

In [35]:
df_final[df_final['pair']==(60979151, 101023204)]

,Product Id,Manufacturer Name,Manufacturer PID,model,Short Description,Long Description,Additional description,score,matched_pid_left,matched_pid_right,matched_brand_left,matched_brand_right,matched_Product Id,matched_model,matched_Manufacturer PID,matched_Supplier PID,matched_Manufacturer Name,matched_Supplier Name,pair
219451,101023204,BALLUFF,BCC02MM,BCC M323-0000-10-001-PX0334-050,Anschlussleitungen,NaN,BKS-S49-1-PU-05; replaced by #BCC M323-0000-10...,100.0,bcc02mm,bcc02mm,balluff,balluff,60979151,BCC M323-0000-10-001-PX0334-05,BCC02MM,NaN,BALLUFF,NaN,"(60979151, 101023204)"


### 3) Intervals for analysis

Analyze values in every interval

In [101]:
intervals = [(i, i + 1) for i in range(90, 100)]
intervals.append((100, 100))

# Count the number of records in each interval
counts = {}
for start, end in intervals:
    if start == end:  # For the last interval (100)
        count = df_final[df_final['score'] == start].shape[0]
    else:
        count = df_final[(df_final['score'] >= start) & (df_final['score'] < end)].shape[0]
    counts[f'{start}-{end}'] = count

for interval, count in counts.items():
    print(f'{interval}: {count}')

90-91: 14897
91-92: 9593
92-93: 4165
93-94: 8411
94-95: 1928
95-96: 690
96-97: 257
97-98: 1
98-99: 0
99-100: 0
100-100: 49615


In [102]:
intervals = [(i, i + 1) for i in range(90, 100)]
intervals.append((100, 100))  # Add the last interval for 100
sampled_data = []

for start, end in intervals:
    if start == end:
        filtered = df_final[df_final['score'] == start]  # Exact match for 100
    else:
        filtered = df_final[(df_final['score'] >= start) & (df_final['score'] < end)]
    
    # Only sample if there is data available
    if not filtered.empty:
        # Sample 100 values (with replacement if needed)
        sampled = filtered.sample(n=100, replace=True) if len(filtered) < 100 else filtered.sample(n=100, replace=False)
        
        # Add the interval as a new column
        sampled['interval'] = f'{start}-{end}'
        sampled_data.append(sampled)

if sampled_data: 
    df_intervals = pd.concat(sampled_data, ignore_index=True)
else:
    df_intervals = pd.DataFrame()  

In [103]:
df_intervals.interval.value_counts()

interval
90-91      100
91-92      100
92-93      100
93-94      100
94-95      100
95-96      100
96-97      100
97-98      100
100-100    100
Name: count, dtype: int64

In [104]:
df_intervals[df_intervals['interval'] == '97-98'].head(2)

,Product Id,Manufacturer Name,Manufacturer PID,model,Short Description,Long Description,Additional description,score,matched_pid_left,matched_pid_right,matched_brand_left,matched_brand_right,matched_Product Id,matched_model,matched_Manufacturer PID,matched_Supplier PID,matched_Manufacturer Name,matched_Supplier Name,pair,interval
700,61249767,Airtree Europe GmbH,NaN,Airtree MX-24 | Schalldämpfer - 38 Typ AirTr...,NaN,NaN,NaN,97.56,airtreemx24schalldaempfer38typairtreemx24,airtreemx24schalldaempfer34typairtreemx24,airtreeeuropegmbh,airtreeeuropegmbh,61249768,Airtree MX-24 | Schalldämpfer-34 Typ Air Tree ...,NaN,NaN,Airtree Europe GmbH,NaN,"(61249767, 61249768)",97-98
701,61249767,Airtree Europe GmbH,NaN,Airtree MX-24 | Schalldämpfer - 38 Typ AirTr...,NaN,NaN,NaN,97.56,airtreemx24schalldaempfer38typairtreemx24,airtreemx24schalldaempfer34typairtreemx24,airtreeeuropegmbh,airtreeeuropegmbh,61249768,Airtree MX-24 | Schalldämpfer-34 Typ Air Tree ...,NaN,NaN,Airtree Europe GmbH,NaN,"(61249767, 61249768)",97-98


Similar most common values

In [108]:
df_final.matched_brand_left.value_counts().head(15)

matched_brand_left
festo                 90453
siemens               20857
balluff               13589
smc                   11206
bosch                  5165
keyence                5082
rockwellautomation     4287
schneiderelectric      3898
fanuc                  3836
ifm                    3717
builttoprint           3476
mahlegmbh              3225
sick                   3169
eaton                  2847
harting                2728
Name: count, dtype: int64

In [109]:
df_intervals.matched_brand_left.value_counts().head(15)

matched_brand_left
festo                 180
balluff               150
siemens               110
airtreeeuropegmbh     100
smc                    31
bosch                  22
ifm                    20
builttoprint           19
fanuc                  17
pilz                   16
rockwellautomation     11
ferramentaria          11
kdac                   11
sick                   10
murrelektronik          9
Name: count, dtype: int64

In [111]:
df_intervals.to_excel('intervals_review.xlsx')